In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
#Importing and pre-processing the images
DATADIR = "Datasets/PetImages"
CATEGORIES = ["Cat", "Dog"]

training_data = []
IMG_SIZE = 70

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data()

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 239 extraneous bytes before marker 0xd9
Corrupt JPEG data: 226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 399 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 254 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2230 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


In [3]:
print(len(training_data))

24946


In [4]:
#Randomizing the data
import random

random.shuffle(training_data)

In [5]:
X = []
y = []

In [6]:
for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y).reshape(-1,1)

In [7]:
X = X/255.0

In [8]:
X_train, X_test = X[5000:], X[:5000]
y_train, y_test = y[5000:], y[:5000]

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

In [ ]:
# Trying out different combinations of layers
dense_layers = [1,2]
layer_sizes = [64, 128, 256]
conv_layers = [1,2,3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = f'dense{dense_layer}-layer_size{layer_size}-conv_layer{conv_layer}-{int(time.time())}'
            tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
            
            model = Sequential()

            model.add(Conv2D(layer_size, (3,3), activation="relu", input_shape = X_train.shape[1:]))
            model.add(MaxPooling2D(pool_size=(2,2), strides=2))

            for l in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3,3), activation="relu"))
                model.add(MaxPooling2D(pool_size=(2,2), strides=2))

            model.add(Flatten())

            for l in range(dense_layer):
                model.add(Dense(layer_size, activation= "relu"))
                model.add(Dropout(0.2))

            model.add(Dense(1, activation= "sigmoid"))

            model.compile(loss="binary_crossentropy", 
                            optimizer="adam", 
                            metrics=["accuracy"])

            model.fit(X_train, y_train, batch_size=32, 
                            epochs=10, 
                            validation_data=(X_test, y_test),
                            callbacks= [tensorboard])

In [10]:
#Best combination for accuracy: 85% achieved
#Convolutional layers=3, Dense layers=1, Layer size = 64

NAME = f'dense1-layer_size64-conv_layer3-{int(time.time())}'
tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
            
model = Sequential()

model.add(Conv2D(64, (3,3), activation="relu", input_shape = X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(64, (3,3), activation="relu",)) 
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(64, (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Flatten())

model.add(Dense(64, activation= "relu"))
model.add(Dropout(0.2))

model.add(Dense(1, activation= "sigmoid"))

model.compile(loss="binary_crossentropy", 
                optimizer="adam", 
                metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=32, 
                epochs=10, 
                validation_data=(X_test,y_test),
                callbacks= [tensorboard])

2022-04-28 13:36:33.544770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 13:36:33.570814: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 13:36:33.571014: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 13:36:33.571340: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the app

Epoch 1/10


2022-04-28 13:36:35.533194: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8303


624/624 [==============================] - 6s 7ms/step - loss: 0.6257 - accuracy: 0.6377 - val_loss: 0.5563 - val_accuracy: 0.7270
Epoch 2/10
624/624 [==============================] - 4s 6ms/step - loss: 0.5173 - accuracy: 0.7464 - val_loss: 0.4943 - val_accuracy: 0.7678
Epoch 3/10
624/624 [==============================] - 4s 6ms/step - loss: 0.4368 - accuracy: 0.7989 - val_loss: 0.4096 - val_accuracy: 0.8124
Epoch 4/10
624/624 [==============================] - 4s 6ms/step - loss: 0.3859 - accuracy: 0.8260 - val_loss: 0.3902 - val_accuracy: 0.8204
Epoch 5/10
624/624 [==============================] - 4s 6ms/step - loss: 0.3387 - accuracy: 0.8500 - val_loss: 0.3799 - val_accuracy: 0.8312
Epoch 6/10
624/624 [==============================] - 4s 6ms/step - loss: 0.2974 - accuracy: 0.8742 - val_loss: 0.3417 - val_accuracy: 0.8502
Epoch 7/10
624/624 [==============================] - 4s 6ms/step - loss: 0.2485 - accuracy: 0.8955 - val_loss: 0.3636 - val_accuracy: 0.8450
Epoch 8/10
624/62

In [11]:
y_predicted = model.predict(X_test)
y_predicted= (y_predicted>0.5)

#Tests to check if used model is good
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(f'Accuracy score is {accuracy_score(y_test, y_predicted)}')
print(f'\nClassification report:\n{classification_report(y_test, y_predicted)}')
print(f'\nConfusion matrix:\n{confusion_matrix(y_predicted, y_test)}')

Accuracy score is 0.8572

Classification report:
              precision    recall  f1-score   support

           0       0.85      0.86      0.85      2420
           1       0.87      0.85      0.86      2580

    accuracy                           0.86      5000
   macro avg       0.86      0.86      0.86      5000
weighted avg       0.86      0.86      0.86      5000


Confusion matrix:
[[2081  375]
 [ 339 2205]]


In [12]:
#Making custom predictions
def make_prediction(test_img):
    t_img = cv2.imread(test_img)
    t_img_rz = cv2.resize(t_img, (IMG_SIZE, IMG_SIZE))
    t_img_rs = t_img_rz.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
    result = model.predict(t_img_rs)
    return result

In [13]:
pred = make_prediction('dog2.jpg')
print(CATEGORIES[int(pred[0][0])])

Dog
